**분석개요**

    A. 여행상품홍보관 광고성과분석 리포트
        1. 매체별 유입 분석
            1) 유입
            2) 재방문
            3) 전환
            4) 이상 유저
            5) 앱설치
            6) 활성

In [1]:
# basic
import gc
import os
import sys
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#--------------------#
# handling
#--------------------#
import math
import time
import random
# import openpyxl
import importlib
import xlsxwriter
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime, timedelta
from scipy.stats import pearsonr
import difflib

#--------------------#
# Vis
#--------------------#
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
## Vis for jupyter theme
from IPython.display import Image
from jupyterthemes import jtplot
jtplot.style(theme= 'grade3', context='notebook', ticks=True, grid=False) ## dark backgroud jupyter notebook theme
# plt.style.use(['seaborn-white']) ## dark background style ## white style : 'seaborn-white'
plt.style.use(['dark_background']) ## dark background style ## white style : 'seaborn-white'
## Vis for korean 
import matplotlib.font_manager as fm
font_location = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'
fprop = fm.FontProperties(fname=font_location)
font_name = fprop.get_name()
matplotlib.rc('font', family=font_name)

In [2]:
#--------------------#
# sphere package
#--------------------#
sys.path.append("/home/das_share/sphere_class/")
import SpherePackage
from SpherePackage import *
for pkg in [SpherePackage] :
    _ = importlib.reload(pkg)

#--------------------#
# kto package
#--------------------#
sys.path.append("../src/")
import kto_config
import kto_util
import kto_prep
from kto_config import *
from kto_prep import *
from kto_util import *
from run_cdj import *

for pkg in [kto_config, kto_prep, kto_util] :
    _ = importlib.reload(pkg)

#-------------------#
# pickle
#-------------------#
sys.path.append("/home/minkyung62/analysis_report/00_custom_analysis/kto_custom/notebook_git/return_pickle.py")
import return_pickle
from return_pickle import *
for pkg in [return_pickle] :
    _ = importlib.reload(pkg)

# 1.Data Import

In [3]:
#------------------------------------------------#
# 1. log Data import
#------------------------------------------------#
CheckDate.check_date_return_pickle(e_date='20221130', s_date='20221020')
df_app_log = ReadFile.read_pickle(e_date='20221130', s_date='20221020')
df_app_log = SpherePrep.BasicPrep.basic_prep(df_app_log)   

# df_app_log_before = df_app_log[df_app_log['date'] < datetime.strptime('20221107', '%Y%m%d')]
# df_app_log_after = df_app_log[df_app_log['date'] >= datetime.strptime('20221107', '%Y%m%d')]

df_app_log.head(3)

,date,duration,sid,seq,app_key,sphere_id,user_id,uid,adid,gender,birth_y,user_grade,created_date,events,referrer,install_referrer,session,abs_events,day
3935505,2022-10-20 00:00:02,2.832,bgON0628_pX07M7zqjQFGm,1,qsJfZhE2Ft85SvZnBo2KSL,jxJXzaDhlSQgbRy1_VDg7yKQj7RKx,,KcL2M2Ab8lS_gd24_KbXbg,,,0,,2022-10-20 00:00:02,"[{'name': 'contentMain', 'timestamp': 16661916...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Spider'}, 'ip': '211.249...",[contentMain],2022-10-20
3935609,2022-10-20 00:00:02,0.705,Er6klS37BPU4FacYHdCxeI,1,qsJfZhE2Ft85SvZnBo2KSL,s7YQZN3kmOOw2Q9Bok1EVWo3nmX3G,,RkCFnnvZi-x-OUhoz8G-6A,,,0,,2022-10-20 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Apple'}, 'ip': '110.70.5...",[sapContentDetail],2022-10-20
3935602,2022-10-20 00:00:02,1680.607,gVACp95v8aSXECgOX6EWNq,1,qsJfZhE2Ft85SvZnBo2KSL,nT4C7Uf3s4_ziY--3_qsWZpngOloj,,uQZs-0CPowzdFzl3BPTRkQ,,,0,,2022-10-20 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '211.11...","[sapContentDetail, subscribeClicsapContentShar...",2022-10-20


# 2.Preprocess

In [4]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
# ## 2) user prop
df_prop = SpherePrep.Prop.df_prop_pipe(df_app_log, KEY_ID = [KEY_ID_DEVICE, KEY_ID_USER], drop_none_prop=False)       # user data

## 3) param - 1
_, df_param_all = DataImport.json_to_dataframe_nodeN(
    df_app_log, [KEY_ID_DEVICE, KEY_ID_USER]
    )

## 4) referr
lst_extrated_referr_cols = ['utm_campaign', 'utm_source', 'utm_medium']
df_app_log = SpherePrep.Attribution_Prep.gen_session_referr_cols(
        df_app_log, 
        referr_col = 'referrer', 
        lst_extrated_referr_cols = lst_extrated_referr_cols,
        fill_na_organic = True)

# 3.Utils

In [5]:
# cnt -> percent function
def cnt_to_percent_ver1(df, col):
    df[col] = df[col].astype(float)

    if  df.index.dtype != 'str':
        df.index = df.index.astype('str')
    
    _lst = []
    for i in range(0,len(df)):
        x = round((df[col][i] / df[col].sum())*100, 2)
        _lst.append(x)

    df['percent']= _lst
    return df


def get_daily_cnt(
                  dict_output, lst, nm, dict_input, df_target = df_app_log, KEY_ID = KEY_ID_DEVICE, 
                  make_sub_dict = False):
    # 1. 전체 일별 수치
    _df_target_daily = df_target[df_target[KEY_ID].isin(lst)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'total_{nm}')

    # 2. 매체별 일별 수지
    for key, values in dict_input.items():
        _lst_id = list(set(lst) &  set(values))
        _df = df_target[df_target[KEY_ID].isin(_lst_id)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'{key}_{nm}').fillna(0)
        if make_sub_dict == True:
            dict_output[f'{key}'] = {}

        dict_output[f'{key}'][f'{nm}'] = _df
        # dict_output[f'{key}_{nm}'] 
        _df_target_daily[f'{key}'] = _df[f'{key}_{nm}']
        _df_target_daily.fillna(0, inplace=True)
        # pd.concat([_df_target_daily, dict_output[f'{key}'][f'{key}_{nm}']], axis=1).fillna(0)
        
    return _df_target_daily

# 4.Analysis
## 0)Segment Setting

In [6]:
#------------------------------------------------#
# 1. segment by referrer
#------------------------------------------------#
dict_lst_user_from_referrs = KtoPrep.get_user_list_by_referr(
    df_app_log, KEY_ID = KEY_ID_DEVICE,
    DEPTH_1_COL = 'referrer_utm_campaign', 
    DEPTH_2_COL = 'referrer_utm_source') 
dict_lst_user_from_referrs_tgpr = dict_lst_user_from_referrs['2022_tgpr']
dict_lst_user_from_referrs_tgpr['organic']  = dict_lst_user_from_referrs['organic']['organic'] # organic 유입 추가

#------------------------------------------------#
# 2. segment by duration per device
#------------------------------------------------#
df_duration_per_device = df_app_log.groupby([KEY_ID_DEVICE])['duration'].sum()
dict_duration_seg = dict()
dict_duration_seg['high'] = df_duration_per_device[
    df_duration_per_device >= df_duration_per_device.quantile(0.66)].index
dict_duration_seg['middle'] = df_duration_per_device[
    (df_duration_per_device < df_duration_per_device.quantile(0.66)) &
    (df_duration_per_device >= df_duration_per_device.quantile(0.33))
    ].index
dict_duration_seg['low'] = df_duration_per_device[
    df_duration_per_device < df_duration_per_device.quantile(0.33)].index

# #------------------------------------------------#
# # 3. segment by new or re
# #------------------------------------------------#
# ## 아래 코드가 실행되지 않아 created_date를 datetime64으로 형변환 시도 
# # df_app_log['created_date'] = pd.DatetimeIndex(df_app_log['created_date'])
# lst_new_device = list(df_app_log[df_app_log['created_date'] >= datetime.strptime('20221020', '%Y%m%d')][KEY_ID_DEVICE].unique())
# lst_re_device = list(df_app_log[df_app_log['created_date'] < datetime.strptime('20221020', '%Y%m%d')][KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 4. segment by fraud or total
#------------------------------------------------#
lst_fraud_device = list(df_duration_per_device[df_duration_per_device < 5].index)
lst_total_device = list(df_app_log[KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 5. segment by demo (gender or age)
#------------------------------------------------#
# ## gender
# dict_seg_gender = dict()
# dict_seg_gender['m'] = list(df_prop[df_prop['gender'] == 'm'][KEY_ID_DEVICE])
# dict_seg_gender['f'] = list(df_prop[df_prop['gender'] == 'f'][KEY_ID_DEVICE])

# ## age
# dict_seg_age = dict()
# # dict_seg_age['20_30'] = list(df_prop[(datetime.today().year - df_prop['birth_y']) < 40][KEY_ID_DEVICE])
# # dict_seg_age['40_50'] = list(df_prop[(datetime.today().year - df_prop['birth_y']) >= 40][KEY_ID_DEVICE])
# # # # ======================================================== #  next version
# dict_seg_age['20_30'] = list(df_prop[df_prop['age'] < 40][KEY_ID_DEVICE])
# dict_seg_age['40_50'] = list(df_prop[df_prop['age'] >= 40][KEY_ID_DEVICE])

## 1)매체 성과 분석
### (1)유입
- 전체 유입
- 일별 유입
    - 전체 유입
    - 신규 유입
    - 기존 유입

In [7]:
#########################################
# 1. 매체별 전체 유입인원
## 전체 / 일별 총 유입인원
#########################################
dict_output_inflow_total= {}
dict_output_inflow_daily= {}

def count_inflow(dict_input, dict_output_total, dict_output_daily, KEY_ID = KEY_ID_DEVICE, lst = list(df_app_log[KEY_ID_DEVICE].unique()), nm = 'inflow'):
    _df_target_daily = df_app_log[
                        df_app_log[KEY_ID].isin(lst)].groupby('day')[KEY_ID]\
                            .nunique().to_frame(name = f'total_{nm}')

    for _referrer, _segment in dict_input.items():
        ## 1) 전체 총 유입인원
        dict_output_total[f'{_referrer}'] = {}
        dict_output_total[f'{_referrer}'][f'{nm}'] = len(set(_segment))

        ## 2) 일별 총 유입인원
        _lst_segment = list(set(lst) &  set(_segment))
        _df = df_app_log[df_app_log[KEY_ID].isin(_lst_segment)].groupby('day')[KEY_ID].nunique().to_frame(name = f'{_referrer}').fillna(0)
        
        dict_output_daily[f'{_referrer}'] = {}
        dict_output_daily[f'{_referrer}'][f'{nm}'] = _df
        _df_target_daily[f'{_referrer}'] = _df[f'{_referrer}']
        _df_target_daily.fillna(0, inplace=True)


    ### 1-2) 전체 총 유입인원 df화
    _df_output = pd.DataFrame.from_dict(dict_output_total, orient='index', columns=['inflow'])
    _df_output = cnt_to_percent_ver1(_df_output, 'inflow')
    _df_output = _df_output.sort_values(by = 'inflow', ascending=False)

    return _df_output, _df_target_daily

_df_inflow_total, _df_inflow_daily = count_inflow(dict_lst_user_from_referrs_tgpr, dict_output_inflow_total, dict_output_inflow_daily)

_df_inflow_total
pd.set_option('max_rows', 100)
_df_inflow_daily

,inflow,percent
organic,2265472.0,99.22
NAVER_GFA,10392.0,0.46
KAKAO_moment,7363.0,0.32


,total_inflow,KAKAO_moment,NAVER_GFA,organic
day,,,,
2022-10-20,67791,5,46.0,63640
2022-10-21,73369,10,43.0,70481
2022-10-22,93591,6,39.0,90464
2022-10-23,83871,2,27.0,78995
2022-10-24,68504,11,27.0,64277
2022-10-25,64629,15,38.0,59633
2022-10-26,60004,16,25.0,56714
2022-10-27,50872,11,33.0,46908
2022-10-28,74168,13,28.0,70620


### (2)재방문
- 방문 빈도별 재방문율
- 주차별 재방문율

In [8]:
#------------------------------------------------#
## 2. 재방문율
#------------------------------------------------#
### 1) 1회 / 2회 이상 / 3회 이상 방문자
dict_output_revisit = dict()

#### count revisit rate
def revisit_rate(dict_input, dict_output):
    for _referrer, _segment in dict_input.items():
        dict_output[_referrer] = dict()
        _df_user_day_cnt = df_app_log[df_app_log[KEY_ID_DEVICE].isin(_segment)].groupby(KEY_ID_DEVICE)['day'].count()

        only_1_visit_rate = round(_df_user_day_cnt.value_counts()[1] / len(_segment), 4)
        re_visit_rate = round(1 - only_1_visit_rate, 4)
        re_visit_over_3_rate = round(1 - _df_user_day_cnt.value_counts()[:2].sum() / len(_segment), 4)
        
        dict_output[_referrer]['only_1_visit_rate'] = only_1_visit_rate
        dict_output[_referrer]['re_visit_rate'] = re_visit_rate
        dict_output[_referrer]['re_visit_over_3_rate'] = re_visit_over_3_rate

    _df_output = pd.DataFrame.from_dict(
        dict_output, orient = 'index'
        ).sort_values(by = 're_visit_rate', ascending=False).reset_index()
    
    return _df_output

df_ouput_revisit = revisit_rate(dict_lst_user_from_referrs_tgpr, dict_output_revisit)

df_ouput_revisit

,index,only_1_visit_rate,re_visit_rate,re_visit_over_3_rate
0,KAKAO_moment,0.8118,0.1882,0.0869
1,NAVER_GFA,0.8700,0.1300,0.0550
2,organic,0.8722,0.1278,0.0614


### (3)전환
: 랜딩페이지에서 KPI 페이지까지의 전환 KPI 분석
- 전환수
- 전환율

In [9]:
#########################################
# 1. 구독완료
## 1) 일별 구독완료 유저 수
## 2) 구독전환율
#########################################
lst_landing_device = list(df_param_all[KEY_ID_DEVICE][df_param_all['name']=='promotionMain'].unique())

dict_output_conversion = {}
df_daily_contest = pd.DataFrame()
df_kpi_conversion = pd.DataFrame(index=[
                                'landing_cnt', 
                                'goal_cnt', 
                                'conversion'])

def calculate_conversion(dict_input, dict_output, KPI, df_output_conversion, KEY_ID = KEY_ID_DEVICE, col_nm = 'name'):
    _df_param_target = df_param_all[df_param_all[col_nm]== KPI]

    df_daily_output = _df_param_target.groupby('day')[KEY_ID].nunique().to_frame(name = 'total_cnt')
    _lst_device = list(_df_param_target[KEY_ID].unique())

    for _referrer, _segment in dict_input.items():
        ### 2) 매체별 일별 콘테스트 페이지 접속 유저 수
        _lst_target = list(set(_lst_device) &  set(_segment))

        _df = _df_param_target[_df_param_target[KEY_ID].isin(_lst_target)].groupby('day')[KEY_ID].nunique().to_frame(name = f'{_referrer}' + f'_{KPI}_daily').fillna(0)

        dict_output[f'{_referrer}'] = {}
        dict_output[f'{_referrer}'][f'{KPI}_daily_cnt'] = _df
        df_daily_output[f'{_referrer}'] = _df[f'{_referrer}' + f'_{KPI}_daily']
        df_daily_output.fillna(0, inplace=True)

        ### 3) 전환율
        landing_cnt = len(_segment)
        kpi_cnt = _df_param_target[_df_param_target[KEY_ID].isin(_segment)][KEY_ID].nunique()
        dict_output[f'{_referrer}'][f'{KPI}_conversion'] = [landing_cnt,
                                                        kpi_cnt, 
                                                        (kpi_cnt/landing_cnt)]
        df_output_conversion[f'{_referrer}'] = dict_output[f'{_referrer}'][f'{KPI}_conversion']

    return df_daily_output, df_output_conversion

df_daily_kpi_cnt, df_kpi_conversion = calculate_conversion(
                                        dict_lst_user_from_referrs_tgpr, dict_output_conversion, 
                                        KPI = 'reserveClick', df_output_conversion=df_kpi_conversion ,KEY_ID= KEY_ID_DEVICE)

pd.set_option('max_rows', 100)
df_daily_kpi_cnt
df_kpi_conversion

,total_cnt,KAKAO_moment,NAVER_GFA,organic
day,,,,
2022-10-20,16,0.0,0.0,16
2022-10-21,22,0.0,0.0,22
2022-10-22,6,0.0,0.0,6
2022-10-23,10,0.0,0.0,10
2022-10-24,13,0.0,0.0,12
2022-10-25,15,0.0,0.0,15
2022-10-26,6,0.0,0.0,6
2022-10-27,19,0.0,0.0,19
2022-10-28,12,0.0,0.0,12


,KAKAO_moment,NAVER_GFA,organic
landing_cnt,7363.000000,10392.000000,2.265472e+06
goal_cnt,167.000000,184.000000,5.880000e+02
conversion,0.022681,0.017706,2.595486e-04


In [10]:
df_daily_kpi_cnt, df_kpi_conversion = calculate_conversion(
                                        dict_lst_user_from_referrs_tgpr, dict_output_conversion, 
                                        KPI = 'promotionDetail', df_output_conversion=df_kpi_conversion ,KEY_ID= KEY_ID_DEVICE)

df_daily_kpi_cnt
df_kpi_conversion

,total_cnt,KAKAO_moment,NAVER_GFA,organic
day,,,,
2022-10-20,58,0.0,0.0,57
2022-10-21,61,0.0,0.0,60
2022-10-22,24,0.0,0.0,24
2022-10-23,31,0.0,0.0,31
2022-10-24,66,0.0,0.0,65
2022-10-25,60,0.0,0.0,60
2022-10-26,56,0.0,0.0,56
2022-10-27,68,0.0,0.0,68
2022-10-28,52,0.0,0.0,52


,KAKAO_moment,NAVER_GFA,organic
landing_cnt,7363.00000,10392.000000,2.265472e+06
goal_cnt,559.00000,424.000000,1.977000e+03
conversion,0.07592,0.040801,8.726658e-04


### (4)Fraud
- 바운스율
- 정착율

In [11]:
#########################################
# 1. 바운스 유저 비율 & 정착 유저 비율
#########################################
dict_fraud_referrs = {}

def count_fraud(dict_input, dict_output, lst_fraud = lst_fraud_device):
    for _referrer, _segment in dict_input.items():
        _lst_device= list(set(_segment) & set(lst_fraud))       ## 수정된 부분
        
        total_cnt = len(_segment)
        fraud_cnt = len(_lst_device)
        settle_cnt = total_cnt - fraud_cnt
        fraud_rate = fraud_cnt / total_cnt
        settle_rate = 1 - fraud_rate                                

        dict_output[f'{_referrer}'] = [total_cnt, fraud_cnt, settle_cnt, fraud_rate, settle_rate]

    _df_output = pd.DataFrame.from_dict(
        dict_output, orient='index', columns=[
            'total_cnt', 'fraud_cnt', 'settle_cnt', 'fraud_rate', 'settle_rate']).sort_values(by = 'fraud_rate', ascending = False)
    
    return _df_output, dict_output

df_fraud_referrs, dict_fraud_referrs = count_fraud(dict_lst_user_from_referrs_tgpr, dict_fraud_referrs)

df_fraud_referrs

,total_cnt,fraud_cnt,settle_cnt,fraud_rate,settle_rate
NAVER_GFA,10392,8239,2153,0.792821,0.207179
organic,2265472,1757017,508455,0.775563,0.224437
KAKAO_moment,7363,5540,1823,0.752411,0.247589


### (5)앱설치 유저

In [12]:
# #########################################
# 0. prep
# #########################################
EVENT_APP_INSTALL = '#appInstall'
app_key_android = 'qsJfZhE2Ft85SvZoXInoo3'
app_key_ios = 'qsJfZhE2Ft85SvZTOvxsUA'

#########################################
# 1. 앱설치 유저
#########################################
dict_install = {}

def count_app_install(dict_input, dict_output):
    _df_log_install = df_app_log[df_app_log['abs_events'].apply(lambda x : True if EVENT_APP_INSTALL in x else False)]

    for _referrer, _segment in dict_input.items():
        ## 1) 캠페인 유저의 사용자 ID 추출
        _lst_campaign_user_id = list(set(df_prop[df_prop[KEY_ID_DEVICE].isin(_segment)][KEY_ID_USER]))
        
        ## 2) 캠페인 유저의 사용자 ID중 android & ios 앱 사용자
        dict_output[f'{_referrer}'] = {}
        _lst_campaign_user_ids =\
            list( 
                df_app_log[
                (df_app_log['app_key'].isin([app_key_android, app_key_ios])) & 
                (df_app_log[KEY_ID_USER].isin(_lst_campaign_user_id))
                ][KEY_ID_USER].unique()
                )
            # _df_log_install[_df_log_install[KEY_ID_USER].isin(_lst_user_id)][KEY_ID_USER]

        dict_output[f'{_referrer}'] = len(_lst_campaign_user_ids)

    df_output = pd.DataFrame.from_dict(
                    dict_output, orient='index', columns= ['install_cnt']).sort_values(by = 'install_cnt', ascending=False)

    return df_output, dict_output

df_install_referrs, dict_install = count_app_install(dict_lst_user_from_referrs_tgpr, dict_install)

df_install_referrs

,install_cnt
organic,8832
KAKAO_moment,16
NAVER_GFA,6


### (6)활성
- 인당 체류 시간
- 인당 컨텐츠 조회수
- 검색 조회 빈도

In [13]:
#########################################
# 1. 인당 체류시간
#########################################
dict_duration = {}
dict_events_cnt_output = {}

def activation(dict_input, dict_ouput1, dict_output2, KEY_ID = KEY_ID_DEVICE):
    lst_device_no_fraud = list(set(lst_total_device) - set(lst_fraud_device))
    _df_log_no_fraud = df_app_log[df_app_log[KEY_ID].isin(lst_device_no_fraud)]

    for _referrer, _segment in dict_input.items():
        ## 1) 평균 체류 시간
        _df = _df_log_no_fraud[_df_log_no_fraud[KEY_ID_DEVICE].isin(_segment)]
        dict_ouput1[f'{_referrer}']= _df.groupby(KEY_ID)['duration'].sum().mean()

        ## 2) 평균 컨텐츠 조회수
        _df_app_log_by_referr = df_app_log[
                                    df_app_log[KEY_ID_DEVICE].isin(_segment)]
        _df_app_log_by_referr['events_cnt'] = _df_app_log_by_referr['abs_events'].apply(lambda x : len(x))

        dict_output2[f'{_referrer}'] = _df_app_log_by_referr.groupby(KEY_ID_DEVICE)['events_cnt'].sum().mean()

    ### 1-2) 평균 체류 시간 df화
    _df_output1 = pd.DataFrame.from_dict(dict_ouput1, orient='index', columns=['average_duration'])
    _df_output1 = _df_output1.sort_values(by = 'average_duration', ascending=False)

    ### 2-2) 평균 컨텐츠 조회수 df화
    _df_output2 = pd.DataFrame.from_dict(dict_output2, orient='index',  columns=['average_event_cnt']).sort_values(by = 'average_event_cnt', ascending=False)

    return _df_output1, _df_output2, lst_device_no_fraud

df_duration, df_event_cnt, lst_device_no_fraud = activation(dict_lst_user_from_referrs_tgpr, dict_duration, dict_events_cnt_output)

df_duration
df_event_cnt

,average_duration
NAVER_GFA,384.124133
organic,359.414812
KAKAO_moment,296.377788


,average_event_cnt
KAKAO_moment,2.915252
NAVER_GFA,2.487587
organic,2.382553
